In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [5]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [6]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [7]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [8]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'I am the guy who usually keeps opinions to himself, but I just got back from this movie, and felt I had to express my opinions. Let me start by saying that I am a HUGE horror fan. But what makes a horror movie? I sure like to see even a tiny bit of a good script and character development. I know they often lack in horror movies, but Prom Night looked like it didn\'t even put forth ANY effort in that department. Next, we all love suspense. That on the edge of your seat suspense with unpredictable surprises. Yeah, Prom Night had none of that! Of course, we like a terrifying killer. Prom Night have that? Nope, it has a pretty boy with a cute lil\' knife. And when all else fails...at least horror has its guilty pleasure to make it enjoyable like gore gore gore, and the occasional nude scene! Yeah, well when you have a horror movie rated PG-13 like Prom Night, they leave that stuff out too. So with all of these elements missing, I ask....does this still count as a horror movie? N

### Buat Teks Encoder

In [9]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [12]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [14]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 10, 237,   2, ...,   0,   0,   0],
       [ 11, 503,  14, ...,   0,   0,   0],
       [ 11,   7,   4, ...,   0,   0,   0]])

In [15]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'I am the guy who usually keeps opinions to himself, but I just got back from this movie, and felt I had to express my opinions. Let me start by saying that I am a HUGE horror fan. But what makes a horror movie? I sure like to see even a tiny bit of a good script and character development. I know they often lack in horror movies, but Prom Night looked like it didn\'t even put forth ANY effort in that department. Next, we all love suspense. That on the edge of your seat suspense with unpredictable surprises. Yeah, Prom Night had none of that! Of course, we like a terrifying killer. Prom Night have that? Nope, it has a pretty boy with a cute lil\' knife. And when all else fails...at least horror has its guilty pleasure to make it enjoyable like gore gore gore, and the occasional nude scene! Yeah, well when you have a horror movie rated PG-13 like Prom Night, they leave that stuff out too. So with all of these elements missing, I ask....does this still count as a horror movie?

In [16]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [18]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [19]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

1/1 [==============================] - 3s 3s/step
[0.00769014]


In [20]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

1/1 [==============================] - 1s 1s/step
[0.00769014]


In [21]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

### Train Model